# <div style="text-align: center" >Multiple Classification - Hyper Parameters Selection</div>

In [0]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import sklearn as sk
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
cd /content/gdrive/My\ Drive/Vahid\ Ghafourian

/content/gdrive/My Drive/Armita Razavi


### Load  data  mat files

In [0]:
# Get temperature data and make it as a list in order to have it as a feature in dataset
dst = loadmat('../datasets/data_trend_tem.mat')
dst_l = dst['trend_tem'].tolist()[0]
# dst['trend_tem']
dst_list = np.asarray(dst_l)
# dst_list

In [0]:
# Load all mat data files in a list format
dataDir = "../datasets/"
mats = []
for file in os.listdir(dataDir):
     if file.endswith(".mat"):
        mats.append(loadmat(dataDir+file))

### Create DataFrames From mat Files

In [0]:
# Convert mat data files to pandas dataframes and give names to the columns
#  Healthy state shown as 1 - Damaged state shown as 0 in the State target

cols = ['Power','Input DG','Output Temp','Output Press','Fuel Debi','TIT','TET']
df = {}

for i in range(len(mats)-1):
    df[i] = pd.DataFrame(list(mats[i].values())[3],columns= cols)
    df[i]['Temperature'] = dst_list
    if i == 0:
        df[i]['State'] = '0'
    elif i == 1:
        df[i]['State'] = '1'
    elif i == 2:
        df[i]['State'] = '2'
    elif i == 3:
        df[i]['State'] = '3'
    elif i == 4:
        df[i]['State'] = '4'
    elif i == 5:
        df[i]['State'] = '5'
    elif i == 6:
        df[i]['State'] = '6'
    elif i == 7:
        df[i]['State'] = '7'
    elif i == 8:
        df[i]['State'] = '8'
    elif i == 9:
        df[i]['State'] = '9'
    elif i == 10:
        df[i]['State'] = '10'
    elif i == 11:
        df[i]['State'] = '11'
    elif i == 12:
        df[i]['State'] = '12'
    elif i == 13:
        df[i]['State'] = '13' 
    
# df[5].head(10)

### Select Samples 

In [0]:
# Select Random Samples From all data for training

sample = {}

for i in range(len(mats)-1):
    if i == 0:
        sample[i] = df[i].sample(n= 600, replace = False, random_state = 0)
    else:    
        sample[i] = df[i].sample(n= 80, replace = False, random_state = 0)   
        
# sample[0].head(10)[['State']]  

### Merge All Samples into One DataFrame and Shuffle the Samples

In [0]:
# Merge all sample data frames to have healthy and damaged samples together
total_sample = sample[0].append([sample[1],sample[2], sample[3],sample[4], sample[5],sample[6], sample[7],
                                 sample[8],sample[9],sample[10],sample[11],sample[12],sample[13]],
                                ignore_index=True)

# Shuffle the total sample data
total_sample = total_sample.sample(frac=1)
# total_sample.head(60)

# type(total_sample)

In [0]:
# Set X and y of the total sample data
X = total_sample[['Power','Input DG','Output Temp','Output Press','Fuel Debi','TIT','TET','Temperature']]
# X = total_sample.iloc[:, 0:8]
y = total_sample['State']
# y = total_sample.lioc[:, 8]



### Create Train and Validation Sets

In [0]:
# split total sample data into test, train and validation sets

# create train set and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.7, random_state=0)

# print train and validation sets proportions
print('train: {}% | validation {}%'.format(round(len(y_train)/len(y),2)*100,round(len(y_val)/len(y),2)*100))
print('train samples: {} | validation samples: {}' .format(X_train.shape[0],X_val.shape[0]))

train: 70.0% | validation 30.0%
train samples: 6650 | validation samples: 2850


### Select Hyper Parameters for Model (Hyper Parameter Configuration)

### K-Nearest Neighbors

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors': [5, 10, 20, 30, 40],
              'weights': ['uniform', 'distance'],
              'algorithm': ['ball_tree', 'kd_tree', 'brute'],
              'leaf_size':[5, 10, 20, 30, 40]
              }

grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=7)

grid.fit(X, y)

print('Best Parameters Chosen for the Model Using GridSearch')
print(grid.best_params_)
print('*********************************************************************')

Model1 = grid.best_estimator_

# y_pred = Model1.fit(X_train, y_train).predict(X_val)

y_preds = cross_val_score(Model1, X, y, cv=5)
print(y_preds)

print('knn accuracy: ', np.mean(y_preds))

# Summary of the predictions made by the classifier
# print(classification_report(y_val, y_pred))
# print(confusion_matrix(y_val, y_pred))

# Predction Score
# print('accuracy is',accuracy_score(y_pred,y_val))
# knn_accuracy = accuracy_score(y_pred,y_val)

Best Parameters Chosen for the Model Using GridSearch
{'algorithm': 'ball_tree', 'leaf_size': 5, 'n_neighbors': 40, 'weights': 'uniform'}
*********************************************************************
[0.31263158 0.31526316 0.31210526 0.30789474 0.31157895]
knn accuracy:  0.3118947368421052


### Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


param_grid = {'n_estimators': np.arange(1,20),
              'max_depth': [2, 4, 6, 8, 10],
              'max_features': ['sqrt','log2', None]}

grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=7)

grid.fit(X, y)

grid.best_params_

Model2 = grid.best_estimator_

print('Best Parameters Chosen for the Model Using GridSearch')
print(grid.best_params_)
print('*********************************************************************')

# y_pred= Model2.fit(X_train, y_train).predict(X_val)

y_preds = cross_val_score(Model2, X, y, cv=5)
print(y_preds)

print('rf accuracy: ', np.mean(y_preds))

# Summary of the predictions made by the classifier
# print(classification_report(y_val, y_pred))
# print(confusion_matrix(y_val, y_pred))

# Predction Score
# print('accuracy is',accuracy_score(y_pred,y_val))
# rf_accuracy = accuracy_score(y_pred,y_val)

Best Parameters Chosen for the Model Using GridSearch
{'max_depth': 10, 'max_features': None, 'n_estimators': 15}
*********************************************************************
[0.52421053 0.51842105 0.51894737 0.54052632 0.51789474]
rf accuracy:  0.5239999999999999
